In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('/opt/ml/dataset/train/train.csv')
valid = pd.read_csv('/opt/ml/dataset/test/test_data.csv')

In [4]:
df = pd.concat([train, valid])
df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
7760,7760,코로나19 방역 조치의 일환으로 국민의 움직임을 통제하려는 정부의 시도를 이탈리아 ...,"{'word': '정부', 'start_idx': 33, 'end_idx': 34,...","{'word': '이탈리아', 'start_idx': 41, 'end_idx': 4...",100,wikitree
7761,7761,선 연구원은 “위식도역류질환치료제인 케이캡이 92억원 판매되면서 2019년 연간 3...,"{'word': '종근당', 'start_idx': 133, 'end_idx': 1...","{'word': '전년', 'start_idx': 143, 'end_idx': 14...",100,wikitree
7762,7762,"한국전기안전공사(사장 조성완)는 8월 1일부로, 3급 간부직원에 대한 승진·이동 인...","{'word': '한국전기안전공사', 'start_idx': 0, 'end_idx'...","{'word': '조성완', 'start_idx': 12, 'end_idx': 14...",100,wikitree
7763,7763,1987년 B. 슈나이더(B. Schneider)에 의해 만들어졌다.,"{'word': 'B. 슈나이더', 'start_idx': 6, 'end_idx':...","{'word': '1987년', 'start_idx': 0, 'end_idx': 4...",100,wikipedia


In [5]:
df['sentence'].to_csv('train_val.txt',index=False)

In [3]:
train['sentence'].to_csv('train.txt', index=False)
valid['sentence'].to_csv('valid.txt', index=False)

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, DataCollatorWithPadding, DataCollatorForLanguageModeling,EarlyStoppingCallback
import torch
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments
import wandb

In [5]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForMaskedLM.from_pretrained('klue/roberta-large')
model.to(device)

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=512
)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='valid.txt',
    block_size=512
)

data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
wandb.login()
wandb.init(
        project="klue",
        entity="chungye-mountain-sherpa",
        name='mlm_roberta',
        group='mlm',
    )


training_args = TrainingArguments(
    output_dir="./kluebert-retrained",
    overwrite_output_dir=True,
    learning_rate=5e-05,
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_steps=100,
    eval_steps=100,
    save_total_limit=2,
    seed=42,
    save_strategy='steps',
    evaluation_strategy='steps',
    gradient_accumulation_steps=4,
    logging_steps=100,
    fp16=True,
    fp16_opt_level='O1',
    report_to="wandb",
    load_best_model_at_end = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

trainer.train()
trainer.save_model("./kluebert-retrained")

/opt/conda/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:120: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ddobokki (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Using amp fp16 backend
***** Running training *****
  Num examples = 32471
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 3795
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


AssertionError: EarlyStoppingCallback requires load_best_model_at_end = True

In [ ]:
# output_dir=args.output_dir,
# per_device_train_batch_size=hp_config['batch_size'],
# per_device_eval_batch_size=hp_config['batch_size'],
# gradient_accumulation_steps=hp_config['gradient_accumulation_steps'],
# learning_rate=hp_config['learning_rate'],
# weight_decay=hp_config['weight_decay'],
# num_train_epochs=hp_config['epochs'],
# logging_dir=args.logging_dir,
# logging_steps=50,
# save_total_limit=2,
# evaluation_strategy=args.eval_strategy,
# eval_steps=50,
# save_steps=50,
# # dataloader_num_workers=4,
# load_best_model_at_end=True,
# fp16=True,
# fp16_opt_level='O1'